# Goal
- Find false innocents, could be through:
    - connections with many criminal nodes?
    - connection to a big criminal hub?

# Assumptions
- People connected to a crime as suspect or suspect/victim will be considered as being guilty of the crime they are connected to
- 

## Node categorisation
- criminal nodes: nodes that have been a suspect in a acrime at least once
- innocent nodes: nodes that have been only victims and/or witness

## Suspect role


# Work plan
Research question: is it possible to detect criminals that were not previously caught? - "criminal" disguised as "innocent"

- Does a strong link prediction between a criminal and an innocent suggests that the innocent might be a criminal in disguise?
- Compare the results from link prediction and community detection
- If they are in the same community? Is that an even stronger indication of a possible criminal?

# Imports

In [ ]:
import pandas as pd
import statistics as stats
import numpy as np

import networkx as nx
from networkx.algorithms import bipartite

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print('seaborn version', sns.__version__)
print('pandas version', pd.__version__)

# Read data

In [ ]:
al = '..//Data//out.moreno_crime_crime'
gender = '..//Data//ent.moreno_crime_crime.person.sex'
name = '..//Data//ent.moreno_crime_crime.person.name'
role = '..//Data//rel.moreno_crime_crime.person.role'

## Adjancency list as DataFrame

In [ ]:
df_al = pd.read_csv(al, sep=" ", names=['person', 'crime'], index_col=False)
df_al['person'] = 'p' + df_al['person'].astype(str)
df_al['crime'] = 'c' + df_al['crime'].astype(str)
df_al.head(3)
df_al.shape

## Gender Dataframe

In [ ]:
df_gender = pd.read_csv(gender, sep=" ", header=None, names=['gender'])
df_gender['person'] = 'p' + df_gender.index.astype(str)
df_gender.head(3)

## Name DataFrame

In [ ]:
df_name = pd.read_csv(name, sep=" ", header=None, names=['name'])
df_name['person'] = 'p' + df_name.index.astype(str)
df_name.head(3)

## Role Dataframe

In [ ]:
df_role = pd.read_csv(role, sep=",", header=None, names=['role'])
# df_role.head(3)
df_role.shape

## Join adjancency list with role

In [ ]:
df_al_roles = df_al.join(df_role)
df_al_roles.head(3)

In [ ]:
df_al_roles[df_al_roles['person'] == 'p336']

In [ ]:
# group bys gives us the same as degree distribution
# df_al_roles.groupby(by='person', dropna=False).count()
# df_al_roles.groupby(by=['crime', 'role'], dropna=False).count()

## Basic data stats

In [ ]:
# the following are used to create the graph
people = df_al['person'].unique()
crimes = df_al['crime'].unique()
roles = df_role['role'].unique()

# print stats
print('Number of people:', len(people))
print('Number of crimes:', len(crimes))
print('Number of roles:', len(roles))
print('Number of edges:', len(df_al_roles))


Breakdown of roles

In [ ]:
df_role.value_counts()

# Make graph

In [ ]:
# G=nx.from_pandas_dataframe(df_al_roles, 0, 'b', ['weight', 'cost'])

In [ ]:
# create networkx graph
G = nx.Graph()

# # add nodes
for i in range(len(people)):
    G.add_node(people[i], name=df_name['name'][i], gender=df_gender['gender'][i], bipartite=0)

for i in range(len(crimes)):
    G.add_node(crimes[i], bipartite=1)

# # add edges
for i in range(len(df_al)):
    G.add_edge(df_al_roles['person'][i], df_al_roles['crime'][i], role=df_al_roles['role'][i])

## Assign node status

In [ ]:
# Code from Dee
# Initialize a dictionary based on people nodes to keep track of all roles per node
p_nodes = {el:[] for el in people}

# Add all the edge attributes to a dictionary of people nodes
for key,value in nx.get_edge_attributes(G, 'role').items():
    for part in key:
        if part in people:
            p_nodes[part].append(value)

# print(p_nodes['p1']) # List of all roles from p1

# Initialize a dictionary to keep track of who is a "criminal"
status = {el:[] for el in people}
    
# Loop through all roles per node, and deem them criminals if ever they have been a suspect
for key in p_nodes:
    if 'Suspect' in p_nodes[key]:
        status[key] = "criminal"
    elif 'Victim Suspect' in p_nodes[key]:
        status[key] = "criminal"
    else:
        status[key] = "innocent"

print(status['p336']) # Verify that p1 is deemed "criminal"

# Convert to pandas df
criminals_df = pd.DataFrame(status.items(), columns=['node', 'criminal_status'])

## Add node status

In [ ]:
# loop through rows in the data frame and add the attribute of Criminal Status
for index, row in criminals_df.iterrows():
    #print(row['node'])
    G.nodes[row['node']]['criminal_status'] = row['criminal_status']

print(nx.get_node_attributes(G, 'criminal_status')['p1']) # check name of person 'p1' = 'Criminal'

In [ ]:
# get top and bottom nodes for projection and plotting
people_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
crime_nodes = set(G) - people_nodes

In [ ]:
# plot only biggest component
# pos = nx.spring_layout(G)
# posB = nx.bipartite_layout(G, people_nodes)
G_draw = nx.draw_spring(G,node_size=5)

## Get degree of all nodes
Node degreee of **people** nodes if the number of crimes they were involved in.  
Node degree of **crime** nodes is the number of people involved in the crime.

In [ ]:
# Creating dict with all node degrees to add as attribute
node_degrees = dict()

# Creating dict for each node type
people_degrees = dict()
crimes_degrees = dict()

# for loop to populate dicts above
for node in G.nodes:
    # print(G.edges(node, data=True))
    node_degrees[node] = G.degree(node)
    if node.startswith('p') == True:
        people_degrees[node] = G.degree(node)
    else:
        crimes_degrees[node] = G.degree(node)

In [ ]:
df_gender = pd.read_csv(gender, sep=" ", header=None, names=['gender'])
df_gender['person'] = 'p' + df_gender.index.astype(str)
df_gender.head(3)

In [ ]:
# Add node degree as node attribute in graph G
nx.set_node_attributes(G, node_degrees, "node_degree")

# and check it worked
# nx.get_node_attributes(G, 'node_degree')

### Make a dataframe including all node attributes

In [ ]:
# code from https://stackoverflow.com/a/50775962
# make pandas dataframe from graph with node attributes
df_G = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df_G
# df_G['criminal_status'].value_counts()

# Projection on people

In [ ]:
G_proj = bipartite.weighted_projected_graph(G, people_nodes, ratio=False)
list(G_proj.edges(data=True))[0:5]

In [ ]:
print(
    'Num. of nodes: {} \nNum. of edges: {} \nIs bipartite? {} \nIs connected? {}'.format(
        G_proj.number_of_nodes(), 
        G_proj.number_of_edges(), 
        nx.is_bipartite(G_proj),
        nx.is_connected(G_proj)
        )
    )

In [ ]:
weights = list(nx.get_edge_attributes(G_proj, 'weight').values())

# plot weights
plt.hist(weights, bins = 10, log=True)
plt.show()

People of interest. Pairs that are connected by 5 crimes.

In [ ]:
edge_w = list(G_proj.edges(data=True))

for i in edge_w:
    if i[2]['weight'] > 2:
        print(i)

In [ ]:
# get adjacency matrix
A = nx.adjacency_matrix(G_proj, weight='weight')
A = A.toarray()

In [ ]:
A[A > 2]

In [ ]:
# plot adjacency matrix
plt.title('Adjacency Matrix')
# plt.imshow(A, cmap='Greys', markersize = 3)
# plt.show()
# plt.figure(figsize=(8,8))
plt.spy(A, markersize = 1)
plt.show()

In [ ]:
sns.heatmap(A)

In [ ]:
# plot graph visualisation
nx.draw_spring(G_proj, node_size=3)

# Link prediction on projected graph

Code from: https://www.networkatlas.eu/exercise.htm?c=20&e=1

In [ ]:
linkpred_pa = list(nx.preferential_attachment(G_proj))

In [ ]:
# Sort in decreasing score
linkpred_pa.sort(key = lambda tup: tup[2], reverse = True)

In [ ]:
# Get the top 10
for edge_score in linkpred_pa[:10]:
   print(edge_score)

Code from: https://www.networkatlas.eu/exercise.htm?c=20&e=2

In [ ]:
# Run the other link prediction methods implemented in networkx
linkpred_ja = list(nx.jaccard_coefficient(G_proj))
linkpred_aa = list(nx.adamic_adar_index(G_proj))
linkpred_ra = list(nx.resource_allocation_index(G_proj))

linkpred_ja.sort(key = lambda tup: tup[2], reverse = True)
linkpred_aa.sort(key = lambda tup: tup[2], reverse = True)
linkpred_ra.sort(key = lambda tup: tup[2], reverse = True)

In [ ]:
# Print top predictions
print("PrefAtt\tJaccard\tAdamAd\tResAll")
for i in range(5):
   print("%s\t%s\t%s\t%s" % (linkpred_pa[i], linkpred_ja[i], linkpred_aa[i], linkpred_ra[i]))

In [ ]:
df_linkpred = pd.DataFrame(linkpred_ra, columns=['node1', 'node2', 'resource_allocation_score'])
df_linkpred['node1_status'] = df_linkpred['node1'].map(status)
df_linkpred['node2_status'] = df_linkpred['node2'].map(status)
# df_linkpred


In [ ]:
df_linkpred[:10]

In [ ]:
# df_linkpred[df_linkpred['resource_allocation_score'] != 0]

In [ ]:
# sns.displot(df_linkpred[df_linkpred['resource_allocation_score'] > .25])

In [ ]:
df_G.iloc[410,:]

In [ ]:
df_G.iloc[282,:]